In [ ]:
import os, sys
import pandas as pd
import numpy as np
import math
from scipy import stats
import matplotlib.pyplot as plt
import subprocess
import json
import itertools
import re

from IPython.display import display, HTML

cc_algorithms = ['lia', 'olia', 'balia', 'wvegas', 'cubic']

In [ ]:
# filenames: ./logs/{topo}/{cc}/{0Mbps-0Mbps}/{0ms-0ms}/{rep}-{hostname}_iperf.txt
base_path = 'logs'
dir_path = './' + base_path + '/{}/{}/{}Mbps-{}Mbps/{}ms-{}ms/{}-'

def read_flow(sender_file_name, receiver_file_name):
    s = pd.read_csv(sender_file_name, sep='\t')
    r = pd.read_csv(receiver_file_name, sep='\t')
    # print(sender_file_name)
    df = pd.merge(s, r, how='outer', on='pkt_id')
    df['latency [s]'] = df['rcv_t [s]'] - df['snd_t [s]']
    df['latency [ms]'] = (df['rcv_t [s]'] - df['snd_t [s]']) * 1000
    df['sec'] = df['rcv_t [s]'] - df.at[(0, 'snd_t [s]')]
    df['disk_sec'] = df['sec'].apply(np.floor)
    df['disk_msec'] = df['sec'].apply(lambda x: np.floor(x * 1000))
    return df

def tp_array(df, bucket_size_ms=100):
    m = int(df['disk_msec'].max() / bucket_size_ms)
    byt, borders, _ = stats.binned_statistic(df['disk_msec'], df['payload [bytes]'], 'sum', bins=[i*bucket_size_ms for i in range(m)])
    ndf = pd.DataFrame({'msec': borders[:-1], 'tp [Mbps]': byt * 0.008 / bucket_size_ms})
    return ndf

def mean_tp(df, cutoff_s=2):
    latest_time = df['disk_msec'].max()
    df = tp_array(df)
    df = df[(df['msec'] >= cutoff_s*1e3) & (df['msec'] < latest_time - cutoff_s*1e3)]
    return df['tp [Mbps]'].mean()

def mean_latency(df, cutoff_s=2):
    latest_time = df['disk_msec'].max()
    df = df[(df['disk_msec'] > cutoff_s * 1000) & (df['disk_msec'] < latest_time - cutoff_s * 1000)]
    return df['latency [ms]'].mean()

# TODO enable single flow and multiflow loading
def load_experiments(topology, cc_algorithms, tps_a, tps_b, delays_a, delays_b, repetitions=3, postfix=''):
    df = pd.DataFrame()
    for cc in cc_algorithms:
        for tp_a in tps_a:
            for tp_b in tps_b:
                for delay_a in delays_a:
                    for delay_b in delays_b:
                        temp = pd.DataFrame()
                        for rep in range(repetitions):
                            # file name without host specified!
                            file_name = dir_path.format(topology, cc, tp_a, tp_b, delay_a, delay_b, rep)
                            # print(file_name + 'h2{}.txt'.format(postfix))
                            # if os.
                            flow1 = read_flow(file_name + 'h1{}.txt'.format(postfix), file_name + 'h2{}.txt'.format(postfix))
                            flow2 = read_flow(file_name + 'h3{}.txt'.format(postfix), file_name + 'h4{}.txt'.format(postfix))
                            temp = temp.append({
                                'cc': cc,
                                'tp_a': tp_a,
                                'tp_b': tp_b,
                                'delay_a': delay_a,
                                'delay_b': delay_b,
                                'rep': rep,
                                'mean_tp_flow1': mean_tp(flow1),
                                'mean_de_flow1': mean_latency(flow1),
                                'mean_tp_flow2': mean_tp(flow2),
                                'mean_de_flow2': mean_latency(flow2)
                            }, ignore_index=True)
                        # FIXME std of means is not good enough, calculate real std
                        df = df.append({
                            'cc': cc,
                            'tp_a': tp_a,
                            'tp_b': tp_b,
                            'delay_a': delay_a,
                            'delay_b': delay_b,
                            'mean_tp_flow1': temp['mean_tp_flow1'].mean(),
                            'std_tp_flow1': temp['mean_tp_flow1'].std(),
                            'mean_de_flow1': temp['mean_de_flow1'].mean(),
                            'std_de_flow1': temp['mean_de_flow1'].std(),
                            'mean_tp_flow2': temp['mean_tp_flow2'].mean(),
                            'std_tp_flow2': temp['mean_tp_flow2'].std(),
                            'mean_de_flow2': temp['mean_de_flow2'].mean(),
                            'std_de_flow2': temp['mean_de_flow2'].std()
                        }, ignore_index=True)
    return df


topologies = ['mptcp-host-pair', 'MPflow_lpkt','MPvsSP', 'MPvsSP_lpkt']
cc_algorithms = ['lia', 'olia', 'balia', 'wvegas', 'cubic']

# Changing Latency Analysis

In [ ]:
# logs/mptcp-host-pair/balia/1ms-1ms/0-h1.txt

topo = 'two_paths'

df = pd.DataFrame([])

delays = np.arange(1, 102, 20)
tp = 10

# df = load_experiments(topo, cc_algorithms, [10], [10], delays, delays)

for cc in cc_algorithms:
    for delay_a in delays:
        for delay_b in delays:
            temp_df = pd.DataFrame()
            for rep in range(1): # TODO reenable repetitions 3
                # file name without host specified!
                file_name = dir_path.format(topo, cc, tp, tp, delay_a, delay_b, rep)
                flow = read_flow(file_name + 'h1.txt', file_name + 'h2.txt')
                temp_df = temp_df.append({
                    'cc': cc,
                    'delay_a': delay_a,
                    'delay_b': delay_b,
                    'rep': rep,
                    'mean_tp': mean_tp(flow),
                    'mean_de': mean_latency(flow)
                }, ignore_index=True)
                # print(flow)
            df = df.append({
                'cc': cc,
                'delay_a': delay_a,
                'delay_b': delay_b,
                'mean_tp': temp_df['mean_tp'].mean(),
                'std_tp': temp_df['mean_tp'].std(),
                'mean_de': temp_df['mean_de'].mean(),
                'std_de': temp_df['mean_de'].std(),
            }, ignore_index=True)

df

In [ ]:
# iperf csv
# timestamp,source_address,source_port,destination_address,destination_port,interval,transferred_bytes,bits_per_second
def read_iperf(sender_file_name, receiver_file_name):
    header_names = ['timestamp','source_address','source_port','destination_address',
                    'destination_port','xxx','interval','transferred_bytes','bits_per_second']
    s = pd.read_csv(sender_file_name, names=header_names, index_col=False)
    s['Mbps'] = s['bits_per_second'] / 1000000
    return s



# TODO: remove when more data is available
def load_iperf_experiments(topology, cc_algorithms, tps_a, tps_b, delays_a, delays_b, repetitions=range(3)):
    df = pd.DataFrame()
    for cc in cc_algorithms:
        for tp_a in tps_a:
            for tp_b in tps_b:
                for delay_a in delays_a:
                    for delay_b in delays_b:
                        temp = pd.DataFrame()
                        for rep in repetitions:
                            # file name without host specified!
                            file_name = dir_path.format(topology, cc, tp_a, tp_b, delay_a, delay_b, rep)
                            flow1 = read_iperf(file_name + 'h1_iperf.csv', file_name + 'h2_iperf.csv')
                            # flow2 = read_iperf(file_name + 'h3_iperf.txt', file_name + 'h4_iperf.txt')
                            temp = temp.append({
                                'mean_tp_flow1': flow1['Mbps'].iloc[-1],
                                # 'mean_tp_flow2': flow2['Mbps'],
                                # 'mean_de_flow2': mean_latency(flow2)
                            }, ignore_index=True)
                        # FIXME: std of means is not good enough, calculate real std
                        df = df.append({
                            'cc': cc,
                            'tp_a': tp_a,
                            'tp_b': tp_b,
                            'delay_a': delay_a,
                            'delay_b': delay_b,
                            'mean_tp_flow1': temp['mean_tp_flow1'].mean(),
                            'std_tp_flow1': temp['mean_tp_flow1'].std(),
                            # 'mean_de_flow1': temp['mean_de_flow1'].mean(),
                            # 'std_de_flow1': temp['mean_de_flow1'].std(),
                            # 'mean_tp_flow2': temp['mean_tp_flow2'].mean(),
                            # 'std_tp_flow2': temp['mean_tp_flow2'].std(),
                            # 'mean_de_flow2': temp['mean_de_flow2'].mean(),
                            # 'std_de_flow2': temp['mean_de_flow2'].std()
                        }, ignore_index=True)
    return df
    
#delays = np.arange(1, 102, 20)
#read_iperf('./logs/two_paths/lia/10Mbps-10Mbps/0ms-0ms/0-h1_iperf.txt', '')

#df_iperf = load_iperf_experiments('mptcp-host-pair', cc_algorithms, [10], [10], delays, delays, repetitions=[0])
#df_iperf

In [ ]:
# from scapy.utils import

def read_iperf_text(sender_file_name, receiver_file_name):
    s = pd.Series()
    # print(sender_file_name)
    with open(sender_file_name, 'r') as f:
        lines = f.read().splitlines()
        sender_line = lines[-4]
        receiver_line = lines[-3]
        # print(sender_line.split())
        # print(receiver_line.split())
        s['client Mbps'] = float(sender_line.split()[-4])
        s['server Mbps'] = float(receiver_line.split()[-3])
    #with open(receiver_file_name, 'r') as f:
    #    lines = f.read().splitlines()
    #    last_line = lines[-1]
    #    # print(last_line.split(' '))
    #    # s['server Mbps'] = float(last_line.split(' ')[-2])
    # print(s)
    return s
    
def load_iperf_experiments_txt(topology, cc_algorithms, tps_a, tps_b, delays_a, delays_b, repetitions=range(3),
                              pcap=False):
    df = pd.DataFrame()
    for cc in cc_algorithms:
        for tp_a in tps_a:
            for tp_b in tps_b:
                for delay_a in delays_a:
                    for delay_b in delays_b:
                        temp = pd.DataFrame()
                        for rep in repetitions:
                            # file name without host specified!
                            file_name = dir_path.format(topology, cc, tp_a, tp_b, delay_a, delay_b, rep)
                            flow1 = read_iperf_text(file_name + 'h1_iperf.csv', file_name + 'h2_iperf.csv')
                            # flow2 = read_iperf(file_name + 'h3_iperf.txt', file_name + 'h4_iperf.txt')
                            
                            # read rtt
                            rtt1 = None
                            if pcap:
                                rtt1 = read_rtt_from_pcap(file_name + 'h1_iperf_dump.pcap')
                            
                            temp = temp.append({
                                'tp_flow1_c': flow1['client Mbps'],
                                'tp_flow1': flow1['server Mbps'],
                                'rtt_flow1': rtt1,
                                # 'mean_tp_flow2': flow2['Mbps'],
                                # 'mean_de_flow2': mean_latency(flow2)
                            }, ignore_index=True)
                        # FIXME: std of means is not good enough, calculate real std
                        # print(temp['tp_flow1'])
                        df = df.append({
                            'cc': cc,
                            'tp_a': tp_a,
                            'tp_b': tp_b,
                            'delay_a': delay_a,
                            'delay_b': delay_b,
                            'mean_tp_flow1': temp['tp_flow1'].mean(),
                            'std_tp_flow1': temp['tp_flow1'].std(),
                            'mean_c_tp_flow1': temp['tp_flow1_c'].mean(),
                            'std_c_tp_flow1': temp['tp_flow1_c'].std(),
                            'mean_rtt_flow1': temp['rtt_flow1'].mean(),
                            # 'mean_de_flow1': temp['mean_de_flow1'].mean(),
                            # 'std_de_flow1': temp['mean_de_flow1'].std(),
                            # 'mean_tp_flow2': temp['mean_tp_flow2'].mean(),
                            # 'std_tp_flow2': temp['mean_tp_flow2'].std(),
                            # 'mean_de_flow2': temp['mean_de_flow2'].mean(),
                            # 'std_de_flow2': temp['mean_de_flow2'].std()
                        }, ignore_index=True)
                break
    return df

delays = np.arange(0, 101, 20)

df_iperf = load_iperf_experiments_txt('two_paths', cc_algorithms, [10], [10], delays, delays, repetitions=[0,1,2])
df_iperf

In [ ]:
def read_rtt_from_pcap(file_name):
    cmd = "tshark -r {} -e tcp.analysis.ack_rtt -T fields"
    a = subprocess.Popen(['tshark', '-r', file_name, '-e', 'tcp.analysis.ack_rtt' ,'-T', 'fields'],
                         stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    out, _ = a.communicate()

    series = pd.Series(out.split(b'\n'))
    series = pd.to_numeric(series, errors='coerce').dropna()
    return series.mean()

# read_rtt_from_pcap('./logs/two_paths/lia/10Mbps-10Mbps/0ms-0ms/0-h1_iperf_dump.pcap')

In [ ]:
def load_config(topology):
    file_name = './topologies/{}.json'.format(topology)
    if not os.path.isfile(file_name):
        print('JSON topology file not found! {}'.format(file_name))
        return None

    with open(file_name, 'r') as f:
        config = json.load(f)

    return config


def get_iperf_pairings(topology):
    config = load_config(topology)
    pairs = []
    for node in [node for node in config['nodes'] if node['id'].startswith('h')]:
        if 'server' in node['properties']:
            pairs.append((str(node['id']), str(node['properties']['server'])))

    # make sure every host is included in some connection
    hosts = itertools.chain.from_iterable(pairs)
    for node in [node for node in config['nodes'] if node['id'].startswith('h')]:
        if node['id'] not in hosts:
            print('Host {} not contained in any host pairings!'.format(node))
    return pairs


def read_iperf_pair_tp(folder, client, server, repetitions):
    file_name = folder + '/{}-{}_iperf.csv'
    goodputs = []
    
    tp_dfs = pd.DataFrame()
    for rep in range(repetitions):
        cli_file = file_name.format(rep, client)
        ser_file = file_name.format(rep, server)
        
        with open(cli_file, 'r') as f:
            content = f.read().replace('iperf Done.', '').strip().splitlines()
            recv_line = content[-1].split()
            send_line = content[-2].split()
            recv_tp = float(recv_line[6])
            send_tp = float(send_line[6])
            # TODO fix unit in experiment script, error here when encountering non standard unit
            if recv_line[7] in 'Kbits/sec':
                recv_tp = recv_tp / 1000
            elif recv_line[7] in 'bits/sec':
                recv_tp = recv_tp / 1000000
            goodputs.append(recv_tp)
            
        with open(ser_file, 'r') as f:
            content = f.read().replace('iperf3: interrupt - the server has terminated', '').strip().splitlines()
            content = content[6:-4] # TODO replace with conditional cutting
            df = pd.DataFrame([l.replace('[', '').replace(']','').split() for l in content],
                             columns=['ID', 'Interval', 'Interval_unit', 'Transfer', 'Transfer_unit', 'Bandwidth', 'Bandwidth_unit'])
            df['repetition'] = rep
            tp_dfs = tp_dfs.append(df, ignore_index=True)
            
    goodputs = pd.Series(goodputs)
    tp_dfs['client'] = client
    tp_dfs['server'] = server
    return (goodputs.mean(), confidence_interval(goodputs)), tp_dfs

def read_iperf_srv_tp_trace(folder, client, server, repetitions):
    file_name = folder + '/{}-{}_iperf.csv'
    goodputs = []
    
    tp_dfs = pd.DataFrame()
    for rep in range(repetitions):
        ser_file = file_name.format(rep, server)
            
        with open(ser_file, 'r') as f:
            content = f.read().replace('iperf3: interrupt - the server has terminated', '').strip().splitlines()
            content = content[6:-4] # TODO replace with conditional cutting
            df = pd.DataFrame([l.replace('[', '').replace(']','').split() for l in content],
                             columns=['ID', 'Interval', 'Interval_unit', 'Transfer', 'Transfer_unit', 'Bandwidth', 'Bandwidth_unit'])
            df['repetition'] = rep
            tp_dfs = tp_dfs.append(df, ignore_index=True)
            
    goodputs = pd.Series(goodputs)
    tp_dfs['client'] = client
    tp_dfs['server'] = server
    return tp_dfs


def confidence_interval(series, z=2.576):
    """
    Calculate confidence interval for a given series. Default is 99% confidence interval.
    See https://en.wikipedia.org/wiki/Confidence_interval#Basic_steps for further values.
    """
    stats = series.agg(['mean', 'count', 'std'])
    return z*stats['std']/math.sqrt(stats['count'])

    
def read_pcap_csv(file_name):
    df = pd.read_csv(file_name, delimiter='\t')
    # put rtt into [ms] instead of [s]
    df['tcp.analysis.ack_rtt'] = df['tcp.analysis.ack_rtt'] * 1000
    return df


def read_iperf_pair_rtt(folder, client, repetitions):
    file_name = folder + '/{}-{}_iperf_dump.csv'
    df = pd.DataFrame()
    for rep in range(repetitions):
        pcap = read_pcap_csv(file_name.format(rep, client))
        pcap['repetition'] = rep
        df = df.append(pcap, ignore_index=True)
    
    rtt_df = df.dropna(subset=['tcp.analysis.ack_rtt'])
    rtt_stats = rtt_df.groupby(['repetition'])['tcp.analysis.ack_rtt'].mean()
    return (rtt_stats.mean(), confidence_interval(rtt_stats)), df


def load_iperf_experiments_new(topology, repetitions=3):
    """
    Read in log data from experiments, every throughput should come out in [Mbps] and times in [ms].
    One exception to this rule is the relative time since experiment start in the rtt trace.
    """
    df = pd.DataFrame()
    pairs = get_iperf_pairings(topology)
    
    df = pd.DataFrame()
    df_trances = pd.DataFrame()

    for dirpath, dirnames, filenames in os.walk('./logs/{}'.format(topology)):
        if dirnames:
            continue
        
        # Leaf folder, read in files and analyze
        # dirpath has form ./logs/two_paths/cubic/25Mbps-9Mbps/10ms-10ms
        dirpath_split = dirpath.split('/') # ['.', 'logs', 'two_paths', 'lia', '9Mbps-13Mbps', '10ms-10ms']
        cc = dirpath_split[-3]
        bw = dirpath_split[-2].split('-')
        de = dirpath_split[-1].split('-')
        
        # read in data for pairings
        row = {'cc': cc}
        for bandwidth, group in zip(bw, ['a', 'b', 'c', 'd']):
            row['bw_' + group] = int(re.sub('[^0-9]', '', bandwidth))
        for delay, group in zip(de, ['a', 'b', 'c', 'd']):
            row['de_' + group] = int(re.sub('[^0-9]', '', delay))
            
        for cli, ser in pairs:
            # TODO handle trace df
            (mean_tp, conf_tp), df_tp_trace = read_iperf_pair_tp(dirpath, cli, ser, repetitions)
            row[ser + '_tp'] = mean_tp
            row[ser + '_tp_conf'] = conf_tp
            
            (mean_rtt, conf_rtt), df_rtt_trace = read_iperf_pair_rtt(dirpath, cli, repetitions)
            row[cli + '_rtt'] = mean_rtt
            row[cli + '_rtt_conf'] = conf_rtt
            
        df = df.append(row, ignore_index=True)
        
    return df

In [ ]:
two_paths_df = load_iperf_experiments_new('two_paths')
shared_link_df = load_iperf_experiments_new('shared_link')

In [ ]:
shared_link_df.head(10)

In [ ]:
def init_plots(df):
    delay_point = 20
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5.5))

    for cc in df['cc'].unique():
        tmp = df[df['cc'] == cc]
        if 'de_b' in tmp.columns:
            tmp = tmp[tmp['de_b'] == delay_point]
        tmp = tmp.sort_values('de_a')
        tmp.plot(x='de_a', y='h2_tp', yerr='h2_tp_conf', label=cc, ax=axes[0], legend=True, grid=True) #, ylim=(0,22))
        tmp.plot(x='de_a', y='h1_rtt', yerr='h1_rtt_conf', label=cc, ax=axes[1], legend=True, grid=True) #, ylim=(0,130))
        # tmp.plot(x='y', y='x', yerr='std_c_tp_flow1', label='', legend=True, grid=True, ylim=(0,22))

    axes[0].set_title('Iperf Throughput')
    axes[0].set_ylabel('Mbps')
    axes[0].set_xlabel('Throughput Mbps')

    axes[1].set_title('Iperf Packet RTT')
    axes[1].set_ylabel('ms')
    axes[1].set_xlabel('ms delay of second link')
    
    if 'de_b' in df.columns:
        axes[0].axvline(delay_point, color='black', ls=':')
        axes[1].axvline(delay_point, color='black', ls=':')
    
init_plots(two_paths_df)

In [ ]:
def plot_rtt_timeline_per_cc(topo, bw_dir, de_dir):
    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(15, 10))
    plt.subplots_adjust(hspace=0.4)
    for i, cc in enumerate(cc_algorithms):
        file = './logs/{}/{}/{}/{}/0-h1_iperf_dump.csv'.format(topo, cc, bw_dir, de_dir)
        df = read_pcap_csv(file)
        df = df.dropna(subset=['tcp.analysis.ack_rtt'])
        df = df[df['tcp.analysis.ack_rtt'] > 0.06]
        
        for key, grp in df.groupby(['tcp.stream']):
            sub = grp.plot(ax=ax[i//2, i%2], style='+', x='frame.time_relative', y='tcp.analysis.ack_rtt', label=key)
        
        sub.title.set_text(cc)
        # plt.legend(loc='best')
        
def plot_tp_timeline_per_cc(topo, bw_dir, de_dir):
    fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(15, 10))
    plt.subplots_adjust(hspace=0.4)
    for i, cc in enumerate(cc_algorithms):
        folder = './logs/{}/{}/{}/{}'.format(topo, cc, bw_dir, de_dir)
        df = read_iperf_srv_tp_trace(folder, 'h1', 'h2', 3)
        print(df.head(10))
        
        for key, grp in df.groupby(['tcp.stream']):
            sub = grp.plot(ax=ax[i//2, i%2], style='+', x='frame.time_relative', y='tcp.analysis.ack_rtt', label=key)
        
        sub.title.set_text(cc)
        # plt.legend(loc='best')
        

#file = './logs/two_paths/lia/10Mbps-10Mbps/40ms-60ms/0-h1_iperf_dump.csv'
#df = read_pcap_csv(file)
#df = df.dropna(subset=['tcp.analysis.ack_rtt'])
#df = df[df['tcp.analysis.ack_rtt'] > 0.06]

#fig, ax = plt.subplots(figsize=(15, 5.5))

#for key, grp in df.groupby(['tcp.stream']):
#    if key == 0.0:
#        continue
#    ax = grp.plot(ax=ax, style='+', x='frame.time_relative', y='tcp.analysis.ack_rtt', label=key)

#plt.legend(loc='best')
# plt.show()

#df.head(10)
plot_rtt_timeline_per_cc('shared_link', '15Mbps', '10ms')
plot_tp_timeline_per_cc('shared_link', '15Mbps', '10ms')

In [ ]:
df_iperf.dropna().groupby(['cc', 'delay_a', 'delay_b'])['mean_rtt_flow1'].mean()

In [ ]:
import matplotlib
import pandas
## comparing different iperf runtimes
delay_point = 40

delays = np.arange(0, 102, 20)

df_iperf = load_iperf_experiments_txt('two_paths', cc_algorithms, [10], [10],
                                      [delay_point], delays, repetitions=[0,1,2])
df_iperf_long = load_iperf_experiments_txt('two_paths_d', cc_algorithms, [10], [10],
                                       [delay_point], delays, repetitions=[0])


x_liperf = df_iperf_long[df_iperf_long['delay_a'] == delay_point]
x_iperf = df_iperf[df_iperf['delay_a'] == delay_point]


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5.5))

for cc in x_liperf['cc'].unique():
    tmp = x_liperf[x_liperf['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp_flow1', label=cc, ax=axes[0], legend=True, grid=True, ylim=(0,22))
    # tmp.plot(x='delay_b', y='mean_c_tp_flow1', label=cc + 'c', ax=axes[0], legend=True, grid=True, ylim=(0,22))

axes[0].set_title('Iperf with tcpdump but no repetitions')
axes[0].axvline(delay_point, color='black', ls=':')
axes[0].set_ylabel('Mbps')
axes[0].set_xlabel('ms delay of second link')

for cc in x_iperf['cc'].unique():
    tmp = x_iperf[x_iperf['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp_flow1', yerr='std_tp_flow1', label=cc, ax=axes[1], legend=True, grid=True, ylim=(0,22))
    # tmp.plot(x='delay_b', y='mean_c_tp_flow1', yerr='std_c_tp_flow1', label=cc + 'c', ax=axes[1], legend=True, grid=True, ylim=(0,22))
    
axes[1].set_title('Iperf test')
axes[1].set_ylabel('Mbps')
axes[1].axvline(delay_point, color='black', ls=':')
axes[1].set_xlabel('ms delay of second link')

In [ ]:
import matplotlib
import pandas
## comparing different iperf runtimes
delay_point = 60

delays = np.arange(0, 102, 30)

#df_iperf = load_iperf_experiments_txt('two_paths', cc_algorithms, [10], [10],
#                                      [delay_point], delays, repetitions=[0,1,2], pcap=True)

x_iperf = df_iperf[df_iperf['delay_a'] == delay_point]


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5.5))

axes[0].set_title('Iperf Throughput')
axes[0].axvline(delay_point, color='black', ls=':')
axes[0].set_ylabel('Mbps')
axes[0].set_xlabel('ms delay of second link')

for cc in x_iperf['cc'].unique():
    tmp = x_iperf[x_iperf['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp_flow1', yerr='std_tp_flow1', label=cc, ax=axes[0], legend=True, grid=True, ylim=(0,22))
    tmp.plot(x='delay_b', y='mean_rtt_flow1', label=cc, ax=axes[1], legend=True, grid=True)
    # tmp.plot(x='delay_b', y='mean_c_tp_flow1', yerr='std_c_tp_flow1', label=cc + 'c', ax=axes[1], legend=True, grid=True, ylim=(0,22))
    
axes[1].set_title('Iperf rtt')
axes[1].set_ylabel('s')
axes[1].axvline(delay_point, color='black', ls=':')
axes[1].set_xlabel('ms delay of second link')

#df_iperf

In [ ]:
## with delay comparison which is not yet working

import matplotlib
import pandas
## comparing different iperf runtimes
delay_point = 40

delays = np.arange(0, 102, 20)

df_iperf = load_iperf_experiments_txt('two_paths', cc_algorithms, [10], [10],
                                      [delay_point], delays, repetitions=[0,1,2])
df_iperf_long = load_iperf_experiments_txt('two_paths_d', cc_algorithms, [10], [10],
                                       [delay_point], delays, repetitions=[0])


x_liperf = df_iperf_long[df_iperf_long['delay_a'] == delay_point]
x_iperf = df_iperf[df_iperf['delay_a'] == delay_point]


fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 5.5))

for cc in x_liperf['cc'].unique():
    tmp = x_liperf[x_liperf['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp_flow1', label=cc, ax=axes[0,0], legend=True, grid=True, ylim=(0,22))
    # tmp.plot(x='delay_b', y='mean_c_tp_flow1', label=cc + 'c', ax=axes[0], legend=True, grid=True, ylim=(0,22))
    tmp.plot(x='delay_b', y='mean_rtt_flow1', label=cc, ax=axes[1,0], legend=True, grid=True, ylim=(0,22))

axes[0,0].set_title('Iperf with tcpdump but no repetitions')
axes[0,0].axvline(delay_point, color='black', ls=':')
axes[0,0].set_ylabel('Mbps')
axes[0,0].set_xlabel('ms delay of second link')

for cc in x_iperf['cc'].unique():
    tmp = x_iperf[x_iperf['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp_flow1', yerr='std_tp_flow1', label=cc, ax=axes[0,1], legend=True, grid=True, ylim=(0,22))
    # tmp.plot(x='delay_b', y='mean_c_tp_flow1', yerr='std_c_tp_flow1', label=cc + 'c', ax=axes[1], legend=True, grid=True, ylim=(0,22))
    # tmp.plot(x='delay_b', y='mean_rtt_flow1', label=cc, ax=axes[1,1], legend=True, grid=True, ylim=(0,22))
    
axes[0,1].set_title('Iperf test')
axes[0,1].set_ylabel('Mbps')
axes[0,1].axvline(delay_point, color='black', ls=':')
axes[0,1].set_xlabel('ms delay of second link')

In [ ]:
## comparing different iperf runtimes
def plot(delay_point=0):
    #delay_point = 20

    delays = np.arange(0, 102, 20)

    df_iperf = load_iperf_experiments_txt('two_paths', cc_algorithms, [10], [10],
                                          [delay_point], delays, repetitions=[0,1,2])
    df_iperf_long = load_iperf_experiments('two_paths_m', cc_algorithms, [10], [10],
                                          [delay_point], np.arange(0, 101, 10), repetitions=[0,1,2])


    x_liperf = df_iperf_long[df_iperf_long['delay_a'] == delay_point]
    x_iperf = df_iperf[df_iperf['delay_a'] == delay_point]

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5.5))

    for cc in x_liperf['cc'].unique():
        tmp = x_liperf[x_liperf['cc'] == cc]
        tmp.plot(x='delay_b', y='mean_tp_flow1', yerr='std_tp_flow1', capsize=5, label=cc, ax=axes[0], legend=True, grid=True, ylim=(0,22))

    axes[0].set_title('Iperf smaller steps')
    axes[0].axvline(delay_point, color='black', ls=':')
    axes[0].set_ylabel('Mbps')
    axes[0].set_xlabel('ms delay of second link')

    for cc in x_iperf['cc'].unique():
        tmp = x_iperf[x_iperf['cc'] == cc]
        tmp.plot(x='delay_b', y='mean_tp_flow1', yerr='std_tp_flow1', capsize=5, label=cc, ax=axes[1], legend=True, grid=True, ylim=(0,22))

    axes[1].set_title('Iperf')
    axes[1].set_ylabel('Mbps')
    axes[1].axvline(delay_point, color='black', ls=':')
    axes[1].set_xlabel('ms delay of second link')

In [ ]:
plot(0)

In [ ]:
plot(ü)

In [ ]:
plot(40)

In [ ]:
plot(80)

In [ ]:
## comparing app with iperf
d = 41

x = df[df['delay_a'] == d]
x_iperf = df_iperf[df_iperf['delay_a'] == d]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5.5))

for cc in x['cc'].unique():
    tmp = x[x['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp', label=cc, ax=axes[0], legend=True, grid=True, ylim=(0,22))

axes[0].set_title('Custom App code')
axes[0].set_ylabel('Mbps')
axes[0].set_xlabel('ms delay of second link')
    
for cc in x_iperf['cc'].unique():
    tmp = x_iperf[x_iperf['cc'] == cc]
    tmp.plot(x='delay_b', y='mean_tp_flow1', label=cc, ax=axes[1], legend=True, grid=True, ylim=(0,22))

axes[1].set_title('Iperf test')
axes[1].set_ylabel('Mbps')
axes[1].set_xlabel('ms delay of second link')

# Changing Throughput Ananlysis

In [ ]:
tps_a = [5, 10, 15]
tps_b = [5, 10, 15]

df = pd.DataFrame()
delay = 10
topo = 'mp-vs-sp'

df1 = load_experiments(topo, cc_algorithms[:-1], tps_a, tps_b, [delay], [delay])
df2 = load_experiments(topo, cc_algorithms[:-1], tps_a, tps_b, [delay], [delay], postfix='big')
df2

In [ ]:
x = df1[df1['tp_a'] == 15]

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

for i, cc in enumerate(x['cc'].unique()):
    for flow in [1, 2]:
        ax = x[x['cc'] == cc].plot(x='tp_b', y='mean_tp_flow{}'.format(flow), yerr='std_tp_flow{}'.format(flow),
                                  label='{} flow {}'.format(cc, flow), ax=axes[i, 0], grid=True, ylim=(0,12))
        ax.set_ylabel('App goodput')
        ax.set_xlabel('Throughput of second path')

        
for i, cc in enumerate(x['cc'].unique()):
    for flow in [1, 2]:
        ax = x[x['cc'] == cc].plot(x='tp_b', y='mean_de_flow{}'.format(flow), yerr='std_de_flow{}'.format(flow),
                              label='{} flow {}'.format(cc, flow), ax=axes[i, 1], grid=True, ylim=(0,120))
        ax.set_ylabel('App goodput')
        ax.set_xlabel('Throughput of second path')


In [ ]:
## compare big with small pkt sizes, top and bottom charts similiar => no difference between sending small and big pkts
x = df1[df1['tp_a'] == 10]

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 7.5))

for cc in x['cc'].unique():
    for flow in [1, 2]:
        x[x['cc'] == cc].plot(x='tp_b', y='mean_tp_flow{}'.format(flow), yerr='std_tp_flow{}'.format(flow),
                              label='{} flow {}'.format(cc, flow), ax=axes[0,flow-1], grid=True, ylim=(0,12))

x = df2[df2['tp_a'] == 10]

for cc in x['cc'].unique():
    for flow in [1, 2]:
        x[x['cc'] == cc].plot(x='tp_b', y='mean_tp_flow{}'.format(flow), yerr='std_tp_flow{}'.format(flow),
                              label='{} flow {}'.format(cc, flow), ax=axes[1,flow-1], grid=True, ylim=(0,12))
